In [1]:
import requests
import json
import csv
import time, datetime,os
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# ###http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20170605&stockNo=2330

# In[12]:

#standard web crawing process
def get_webmsg (year, month, stock_id):
    date = str (year) + "{0:0=2d}".format(month) +'01' ## format is yyyymmdd
    sid = str(stock_id)
    url_twse = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date='+date+'&stockNo='+sid
    try:
        res = requests.post(url_twse)
    except:
        print("Connection refused by the server..")
    finally:
        time.sleep(5)
    soup = bs(res.text, 'html.parser')
    smt = json.loads(soup.text)     #convert data into json
    return smt


# In[13]:

def write_csv(stock_id,directory,filename,smt) :
    writefile = directory + filename               #set output file name
    outputFile = open(writefile,'w',newline='')
    outputWriter = csv.writer(outputFile)
    head = ''.join(smt['title'].split())
    a = [head,""]
    outputWriter.writerow(a)
    outputWriter.writerow(smt['fields'])
    for data in (smt['data']):
        outputWriter.writerow(data)

    outputFile.close()


# In[14]:

#create a directory in the current one doesn't exist
def makedirs (year, month, stock_id):
    sid = str(stock_id)
    yy      = str(year)
    mm       = str(month)
    directory = 'D:/stock'+'/'+sid +'/'+ yy
    if not os.path.isdir(directory):
        os.makedirs (directory)  # os.makedirs able to create multi folders

In [3]:
def get_stock_infos(stock_id):
    dt = datetime.datetime.now()
    df = None
    for y in range(2): ## 2年
        for m in range(12): ## 12個月
            date = str(dt.year-1+y) + "{0:0=2d}".format(m+1) +'01' ## format is yyyymmdd
            url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=' + date + '&stockNo=' + str(stock_id)
            try:
                res = requests.post(url)
            except:
                raise RuntimeError("Connection refused by the server..")
            finally:
                time.sleep(5)
            print(date, res.text)
            soup = bs(res.text , 'html.parser')
            data = json.loads(soup.text)     #convert data into json
            if df is None:
                df = pd.DataFrame(data['data'], columns=data['fields'])
            else:
                df = pd.concat([df, pd.DataFrame(data['data'], columns=data['fields'])], axis=0, ignore_index=True)

            if y==1 and (m+1)==dt.month:
                return df
    return
            

In [4]:
df = get_stock_infos(2454)
# df = get_webmsg(2021, 8, 2330)
df

RuntimeError: Connection refused by the server..

In [23]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df['日期'], open=df['開盤價'], high=df['最高價'], low=df['最低價'], close=df['收盤價'],
                                     increasing_line_color='red', decreasing_line_color='green'),
                      go.Scatter(x=df_ma['日期'], y=df_ma['MA5'], line=dict(color='orange', width=1)),
                      go.Scatter(x=df_ma['日期'], y=df_ma['MA10'], line=dict(color='blue', width=1)),
                      go.Scatter(x=df_ma['日期'], y=df_ma['MA20'], line=dict(color='cyan', width=1))])

fig.show()

In [ ]:
for stock_id in id_list:
    for year in year_list:
        for month in month_list:
            if (dt.year == year and month > dt.month) :break  # break loop while month over current month
            sid = str(stock_id)
            yy  = str(year)
            mm  = month
            directory = 'D:/stock'+'/'+sid +'/'+yy +'/'       #setting directory
            filename = str(yy)+str("%02d"%mm)+'.csv'          #setting file name
            smt = get_webmsg(year ,month, stock_id)           #put the data into smt 
            makedirs (year, month, stock_id)                  #create directory function
            write_csv (stock_id,directory, filename, smt)    # write files into CSV
            time.sleep(1)